Ways to measure the accuracy we use  - to judge how good the model is 

1.Training data & Test Data - we used to divide data in training & testing & testing, while creating model we used training 

& while predicting we used testing data. based on accuracy of two we came to know how good the model is 



2.Cross Validation:

divide the data in parts - - upto you how many parts you want  - KFOLD - no of split

Exact division of rows - no repetition 

eg Data - Data1,  Data2, Data3, Data4, Data5

model1: Data1,  Data2, Data3, Data4  :::: Tested - Data5     # Accuracy =92

model2: Data1,  Data2, Data3, Data5  :::: Tested - Data4     # Accuracy =90

model3: Data1,  Data2, Data4, Data5  :::: Tested - Data3     # Accuracy =88

model4: Data1,  Data3, Data4, Data5  :::: Tested - Data2     # Accuracy =82

model5: Data2,  Data3, Data4, Data5  :::: Tested - Data1     # Accuracy =80
    
overall accuracy ( takes the average)= (92+90+88+82+80)/5

model1 RF or Bagging Model is created by further dividing data :

i.e model1: Data1,  Data2, Data3, Data4  :::: Tested - Data5     # Accuracy =92

model1 : Training Data1,  Data2, Data3, Data4

* Now for RF : out of this data - select random columns to create the DTs
* for Bagging : out of this data - select random rows to create the DTs

Kfold & cross val is for splitting the data 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# dividing the data to create the multiple Decision Tree

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [4]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

#######   Ensemble Technique    #######

# for Bagging
from sklearn.ensemble import BaggingClassifier

# for Random Forest
from sklearn.ensemble import RandomForestClassifier

# for Boosting 
from sklearn.ensemble import AdaBoostClassifier

#for voting 
from sklearn.ensemble import VotingClassifier

In [6]:
filename = 'pima-indians-diabetes.data.csv'
column_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

pima = pd.read_csv(filename,names=column_names)

In [ ]:
pima.head()

In [7]:
x = pima.iloc[:,0:8]
y = pima['class']

In [8]:
# data is divided in 10 parts - 10 models

kfold = KFold(n_splits=10)

In [9]:
# creating the decision Tree Model - Base Model - On Top of it we create Bagging,Random Forest

d_tree = DecisionTreeClassifier()

## Bagged Decision Trees for Classification

    * Same Training Data - Divide equally ( accuracy of all model should not be same likewise divide)
    * Randomly Select the rows - sample with repetition
    * Create Multiple DT Model 
    * Compare all DT for result - By Voting
    

In [10]:
# creating a Bagging Model
n_trees = 100
model_bagging = BaggingClassifier(base_estimator = d_tree,
                                  n_estimators = n_trees,
                                  max_samples = 0.8,
                                  random_state = 8)

In [11]:
# finding the Cross Validation Score of all the models

results_bag = cross_val_score(model_bagging, x, y, cv = kfold)

In [12]:
# we will get the 10 accuracy for 10 models - 10 fold/ 10 split - 10 DT Model
results_bag

array([0.67532468, 0.84415584, 0.72727273, 0.63636364, 0.80519481,
       0.80519481, 0.85714286, 0.83116883, 0.71052632, 0.73684211])

In [13]:
print(results_bag.mean())

0.7629186602870812


## Random Forest Decision Trees for Classification

    * Same Training Data - Divide equally ( accuracy of all model should not be same likewise divide)
    * Randomly Select the COLUMNS - sample with no repetition
    * no of columns = no of features - generally (total features/3)
    * Create Multiple DT Model 
    * Compare all DT for result - By Voting
    

In [14]:
# RF Model Frame
n_trees = 100

model_rf = RandomForestClassifier(n_estimators=n_trees,
                                  max_features=3,
                                  random_state=8)

In [15]:
# calculating the Cross Validation score - accuracy of Created DT - 10 nos

result_rf = cross_val_score(model_rf, x, y, cv = kfold)
result_rf

array([0.7012987 , 0.84415584, 0.75324675, 0.66233766, 0.80519481,
       0.80519481, 0.85714286, 0.85714286, 0.69736842, 0.77631579])

In [16]:
print(result_rf.mean())

0.7759398496240603


### By Random Forest getting feature importance 


In [17]:
# creating  a model
model_rf.fit(x,y)

RandomForestClassifier(max_features=3, random_state=8)

In [18]:
# GETTING  feature importance 

model_rf.feature_importances_

array([0.07297682, 0.28504742, 0.08337574, 0.06069903, 0.06118524,
       0.17498092, 0.12582072, 0.13591412])

In [19]:
model_rf.feature_names_in_

array(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'],
      dtype=object)

In [ ]:
# these are the important features 

## Boosting Decision Trees for Classification

    * Eg. final exam - preliminary exam taken to know where we are
    * m1 - m2 - m3 at each stage we improve model by reducing the 
    * loss = weightage x indicator 
    * new wt = old wt x e ^( learning rate x E)
    * E = 0.5 log[(1-loss)/loss]

In [20]:
n_trees = 10

model_boost = AdaBoostClassifier(base_estimator=d_tree,
                                 n_estimators=n_trees,
                                 learning_rate=1,
                                 random_state=7)

results_boost = cross_val_score(model_boost, x, y, cv = kfold)

In [21]:
results_boost

array([0.64935065, 0.76623377, 0.67532468, 0.61038961, 0.68831169,
       0.75324675, 0.72727273, 0.79220779, 0.64473684, 0.65789474])

In [22]:
print(results_boost.mean())

0.6964969241285031


## Stacking 

    * Multiple Model
    * Model can be - from DT,Logistic regression, RF,Bagging,boosting SVM
    * Majority is selected

In [24]:
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression

In [25]:
# create the sub models

estimators = []

model1 = LogisticRegression(max_iter=500)
estimators.append(('logistic',model1))

model2 = DecisionTreeClassifier()
estimators.append(('DecisionTree',model2))

model3 = SVC()
estimators.append(('SVM',model3))

In [26]:
# create the ensemble model
ensemble = VotingClassifier(estimators)

In [28]:
result_voting = cross_val_score(ensemble,x,y,cv=kfold)

In [29]:
result_voting

array([0.63636364, 0.81818182, 0.72727273, 0.66233766, 0.79220779,
       0.80519481, 0.84415584, 0.85714286, 0.76315789, 0.77631579])

In [30]:
result_voting.mean()

0.7682330827067669